In [ ]:
import pandas as pd
import json

# Carregar o arquivo CSV
file_path = "RESULTADOS EXPERIMENTOS ARTIGO LLM - Página1.csv"  # Substitua pelo caminho do seu arquivo
data = pd.read_csv(file_path)

# PROMPT 1

In [12]:
# Função para processar o JSON do prompt 1 e gerar o ranking
def process_prompt_1_ranking(row):
    debate_id = row['debate']
    model = row['modelo']
    prompt_json = row['prompt_1']

    try:
        # Carregar o JSON
        prompt_data = json.loads(prompt_json)
        debaters = prompt_data['debaters']

        # Gerar o ranking com base no overall_score
        ranking = sorted(debaters, key=lambda x: x['overall_score'], reverse=True)

        # Adicionar posição no ranking com tratamento de empates
        ranked_list = []
        position = 1
        for i, debater in enumerate(ranking):
            if i > 0 and debater['overall_score'] == ranking[i - 1]['overall_score']:
                # Se a pontuação for a mesma que a do debatedor anterior, mantém a mesma posição
                ranked_list.append({"position": ranked_list[-1]['position'], "name": debater['name'], "score": debater['overall_score']})
            else:
                # Caso contrário, atribui a próxima posição
                ranked_list.append({"position": position, "name": debater['name'], "score": debater['overall_score']})
                position += 1  # Incrementa a posição para o próximo debatedor

        # Retornar o resultado como dicionário
        return {
            "debate_id": debate_id,
            "model": model,
            "ranking": ranked_list
        }
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Erro ao processar debate {debate_id} e modelo {model}: {e}")
        return None

# Processar todos os dados do prompt_1
rankings = data.apply(process_prompt_1_ranking, axis=1)

# Filtrar resultados válidos
rankings = [r for r in rankings if r is not None]

# Salvar os rankings em um arquivo
output_path = "rankings_prompt_1.json"
with open(output_path, "w") as outfile:
    json.dump(rankings, outfile, indent=4)

print(f"Rankings gerados e salvos em {output_path}")


Rankings gerados e salvos em rankings_prompt_1.json


# PROMPT 2

In [15]:
# Função para processar o JSON do prompt 2 e gerar o ranking com tratamento de empates
def process_prompt_2_ranking(row):
    debate_id = row['debate']
    model = row['modelo']
    prompt_json = row['prompt_2']

    try:
        # Carregar o JSON
        prompt_data = json.loads(prompt_json)
        debaters = prompt_data['debaters']

        # Gerar o ranking com base no overall_score (ordem decrescente)
        ranking = sorted(debaters, key=lambda x: x['overall_score'], reverse=True)

        # Adicionar posição no ranking, tratando empates
        ranked_list = []
        current_position = 1  # Inicia a posição
        for i, debater in enumerate(ranking):
            if i > 0 and ranking[i]['overall_score'] == ranking[i - 1]['overall_score']:
                # Empate: mantém a posição do debatedor anterior
                ranked_list.append({
                    "position": ranked_list[-1]['position'],
                    "name": debater['name'],
                    "score": debater['overall_score']
                })
            else:
                # Sem empate: atribui uma nova posição
                ranked_list.append({
                    "position": current_position,
                    "name": debater['name'],
                    "score": debater['overall_score']
                })
            current_position += 1

        # Retornar o resultado como dicionário
        return {
            "debate_id": debate_id,
            "model": model,
            "ranking": ranked_list
        }
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Erro ao processar debate {debate_id} e modelo {model}: {e}")
        return None

# Processar todos os dados do prompt_2
rankings = data.apply(process_prompt_2_ranking, axis=1)

# Filtrar resultados válidos
rankings = [r for r in rankings if r is not None]

# Salvar os rankings em um arquivo
output_path = "rankings_prompt_2.json"
with open(output_path, "w") as outfile:
    json.dump(rankings, outfile, indent=4)

print(f"Rankings gerados e salvos em {output_path}")


Rankings gerados e salvos em rankings_prompt_2.json


# PROMPT 3

In [16]:
# Função para processar o JSON do prompt 3 e gerar o ranking com tratamento de empates
def process_prompt_3_ranking(row):
    debate_id = row['debate']
    model = row['modelo']
    prompt_json = row['prompt_3']

    try:
        # Carregar o JSON
        prompt_data = json.loads(prompt_json)
        debaters = prompt_data['debaters']

        # Função para calcular a nota do debatedor
        def calculate_score(debater):
            positive_count = sum([len(debater['positive_events'].get(event, [])) for event in debater['positive_events']])
            negative_count = sum([len(debater['negative_events'].get(event, [])) for event in debater['negative_events']])
            return positive_count - negative_count

        # Gerar o ranking com base na contagem de eventos positivos e negativos
        ranking = sorted(debaters, key=lambda x: calculate_score(x), reverse=True)

        # Adicionar posição no ranking, tratando empates
        ranked_list = []
        current_position = 1  # Inicia a posição
        for i, debater in enumerate(ranking):
            score = calculate_score(debater)

            if i > 0 and score == calculate_score(ranking[i - 1]):
                # Empate: mantém a posição do debatedor anterior
                ranked_list.append({
                    "position": ranked_list[-1]['position'],
                    "name": debater['name'],
                    "score": score
                })
            else:
                # Sem empate: atribui uma nova posição
                ranked_list.append({
                    "position": current_position,
                    "name": debater['name'],
                    "score": score
                })
            current_position += 1

        # Retornar o resultado como dicionário
        return {
            "debate_id": debate_id,
            "model": model,
            "ranking": ranked_list
        }
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Erro ao processar debate {debate_id} e modelo {model}: {e}")
        return None

# Processar todos os dados do prompt_3
rankings = data.apply(process_prompt_3_ranking, axis=1)

# Filtrar resultados válidos
rankings = [r for r in rankings if r is not None]

# Salvar os rankings em um arquivo
output_path = "rankings_prompt_3.json"
with open(output_path, "w") as outfile:
    json.dump(rankings, outfile, indent=4)

print(f"Rankings gerados e salvos em {output_path}")


Erro ao processar debate 8 e modelo Gemini: Expecting ',' delimiter: line 10 column 11 (char 503)
Rankings gerados e salvos em rankings_prompt_3.json


# PROMPT 4

In [17]:

import re

# Função para processar o JSON do prompt 4 e gerar o ranking com tratamento de empates
def process_prompt_4_ranking(row):
    debate_id = row['debate']
    model = row['modelo']
    prompt_json = row['prompt_4']

    try:
        # Carregar o JSON
        prompt_data = json.loads(prompt_json)
        debaters = prompt_data['debaters']

        # Função para calcular a soma das ocorrências de cada debatedor
        def calculate_score(debater):
            total_score = 0

            # Iterar sobre os aspectos de avaliação
            for aspect in debater['evaluation_aspects'].values():
                # Para cada ocorrência dentro de um aspecto, extrair o valor (positivo ou negativo) entre parênteses
                for occurrence in aspect.get('occurrences', []):
                    match = re.search(r'(\+?\-?\d+)', occurrence)
                    if match:
                        total_score += int(match.group(1))
            
            return total_score

        # Gerar o ranking com base na soma das pontuações
        ranking = sorted(debaters, key=lambda x: calculate_score(x), reverse=True)

        # Adicionar posição no ranking, tratando empates
        ranked_list = []
        current_position = 1  # Inicia a posição
        for i, debater in enumerate(ranking):
            score = calculate_score(debater)

            if i > 0 and score == calculate_score(ranking[i - 1]):
                # Empate: mantém a posição do debatedor anterior
                ranked_list.append({
                    "position": ranked_list[-1]['position'],
                    "name": debater['name'],
                    "score": score
                })
            else:
                # Sem empate: atribui uma nova posição
                ranked_list.append({
                    "position": current_position,
                    "name": debater['name'],
                    "score": score
                })
            current_position += 1

        # Retornar o resultado como dicionário
        return {
            "debate_id": debate_id,
            "model": model,
            "ranking": ranked_list
        }
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Erro ao processar debate {debate_id} e modelo {model}: {e}")
        return None

# Processar todos os dados do prompt_4
rankings = data.apply(process_prompt_4_ranking, axis=1)

# Filtrar resultados válidos
rankings = [r for r in rankings if r is not None]

# Salvar os rankings em um arquivo
output_path = "rankings_prompt_4.json"
with open(output_path, "w") as outfile:
    json.dump(rankings, outfile, indent=4)

print(f"Rankings gerados e salvos em {output_path}")


Rankings gerados e salvos em rankings_prompt_4.json


# SELF ASSESSMENT

In [21]:

file_path = "self-assessment.csv"  # Substitua pelo caminho do seu arquivo
data2 = pd.read_csv(file_path)

In [22]:
# Função para processar os votos e gerar o ranking
def process_self_assessment_ranking(row):
    debate_id = row['debate_id']
    self_assessment_winner = row['self_assessment_winner']
    
    # Votos de cada debatedor
    votes = {
        "DEBATEDOR_1": row['debater_1_votes'],
        "DEBATEDOR_2": row['debater_2_votes'],
        "DEBATEDOR_3": row['debater_3_votes'],
        "DEBATEDOR_4": row['debater_4_votes'],
        "DEBATEDOR_5": row['debater_5_votes'],
    }
    
    # Criar ranking baseado na quantidade de votos
    sorted_votes = sorted(votes.items(), key=lambda x: x[1], reverse=True)
    
    # Identificar a posição de cada debatedor, com empates
    ranked_list = []
    position = 1
    last_score = None
    
    for i, (debater, score) in enumerate(sorted_votes):
        if last_score is None or score != last_score:
            # Atualiza a posição apenas quando houver uma mudança no score
            position = i + 1
        ranked_list.append({
            "position": position,
            "name": debater,
            "score": score
        })
        last_score = score  # Atualiza o último score para comparação de empate
    
    # Adicionar os debatedores com 0 votos como "últimos"
    for debater, score in votes.items():
        if score == 0 and debater not in [d['name'] for d in ranked_list]:
            ranked_list.append({
                "position": len(ranked_list) + 1,
                "name": debater,
                "score": score
            })
    
    # Retornar o resultado como dicionário
    return {
        "debate_id": debate_id,
        "ranking": ranked_list
    }

# Processar todos os dados do self_assessment
rankings = data2.apply(process_self_assessment_ranking, axis=1)

# Filtrar resultados válidos
rankings = [r for r in rankings if r is not None]

# Salvar os rankings em um arquivo JSON
output_path = "rankings_self_assessment.json"
with open(output_path, "w") as outfile:
    json.dump(rankings, outfile, indent=4)

print(f"Rankings gerados e salvos em {output_path}")



Rankings gerados e salvos em rankings_self_assessment.json
